In [1]:
import os
import pandas as pd
import re
import cv2
from sklearn.utils import shuffle
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import skimage

import warnings
warnings.filterwarnings('ignore')

from keras.datasets import mnist
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split
from keras.models import load_model

Using TensorFlow backend.


In [2]:
nums = re.compile(r'(\d+)')
def key(value):
    parts = nums.split(value)
    parts[1::2] = map(int, parts[1::2])
    return parts

In [3]:
trainingImages = []
def add_images(df, directory, val):
    for i in sorted(os.listdir(directory),key=key):
        imgFile = directory+i
        img = cv2.imread(imgFile)
        img = cv2.resize(img,(64,64))
        img = img.astype('float32')
        img = img/255.
        trainingImages.append(img)
        new_row = {'image': img, 'yes/no': val}
        df = df.append(new_row, ignore_index=True)
    return df

In [4]:
def sum(list):
    s = 0
    for val in list:
        s += abs(val)
    return s

In [5]:
def accuracy(a, b):
    s = sum(a - b)
    acc = (len(b) - s) / len(b)
    return acc

In [6]:
##### generating dataframe with required stuff

df = pd.DataFrame(columns = ['image', 'yes/no'])

## first the yes images are added

directory = "/home/nahush/Desktop/archive/yes/"
df = add_images(df, directory, 1)

## first the no images

directory = "/home/nahush/Desktop/archive/no/"
df = add_images(df, directory, 0)

## shuffle the resultant dataframe

df = shuffle(df)

## create csv file of this dataframe

df.to_csv('tumour_dataset.csv', index=False)

In [7]:
## generating the data into x and y variables

x = np.asarray(trainingImages)
y = df['yes/no'].to_numpy() 

In [8]:
## splitting the data
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.01, random_state=42)

x_train.shape, y_train.shape

((738, 64, 64, 3), (738,))

In [9]:
EPOCHS = 11
NUM_CLASSES = 2 # (0, 1)

In [10]:
# One hot encoding
y_train = np_utils.to_categorical(y_train)

x_train.shape, y_train.shape

((738, 64, 64, 3), (738, 2))

In [11]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(64,64,3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(64, (3, 3),activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(128, (5, 5), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(NUM_CLASSES, activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='adamax', metrics=['accuracy'])






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [12]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 31, 31, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 29, 29, 64)        18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 27, 27, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 64)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 13, 13, 64)       

In [13]:
model.fit(x_train, y_train, validation_split = 0.1, validation_data = None, shuffle = True, epochs = EPOCHS, batch_size = 40)

Train on 664 samples, validate on 74 samples
Epoch 1/11
664/664 [==============================] - 3s 4ms/step - loss: 0.7077 - acc: 0.6114 - val_loss: 0.6635 - val_acc: 0.6622
Epoch 2/11
664/664 [==============================] - 2s 3ms/step - loss: 0.6710 - acc: 0.6160 - val_loss: 0.6644 - val_acc: 0.6622
Epoch 3/11
664/664 [==============================] - 2s 3ms/step - loss: 0.6713 - acc: 0.6160 - val_loss: 0.6572 - val_acc: 0.6622
Epoch 4/11
664/664 [==============================] - 2s 3ms/step - loss: 0.6640 - acc: 0.6160 - val_loss: 0.6462 - val_acc: 0.6622
Epoch 5/11
664/664 [==============================] - 2s 3ms/step - loss: 0.6634 - acc: 0.6160 - val_loss: 0.6661 - val_acc: 0.6622
Epoch 6/11
664/664 [==============================] - 2s 3ms/step - loss: 0.6699 - acc: 0.6160 - val_loss: 0.6568 - val_acc: 0.6622
Epoch 7/11
664/664 [==============================] - 2s 3ms/step - loss: 0.6662 - acc: 0.6160 - val_loss: 0.6465 - val_acc: 0.6622
Epoch 8/11
664/664 [===========

In [14]:
model.fit(x_train, y_train, validation_split = 0.1, validation_data = None, shuffle = True, epochs = 15 , batch_size = 40)

Train on 664 samples, validate on 74 samples
Epoch 1/15
664/664 [==============================] - 2s 3ms/step - loss: 0.6599 - acc: 0.6160 - val_loss: 0.6431 - val_acc: 0.6622
Epoch 2/15
664/664 [==============================] - 2s 3ms/step - loss: 0.6604 - acc: 0.6160 - val_loss: 0.6417 - val_acc: 0.6622
Epoch 3/15
664/664 [==============================] - 2s 3ms/step - loss: 0.6580 - acc: 0.6160 - val_loss: 0.6418 - val_acc: 0.6622
Epoch 4/15
664/664 [==============================] - 2s 3ms/step - loss: 0.6551 - acc: 0.6160 - val_loss: 0.6475 - val_acc: 0.6622
Epoch 5/15
664/664 [==============================] - 2s 3ms/step - loss: 0.6536 - acc: 0.6160 - val_loss: 0.6365 - val_acc: 0.6622
Epoch 6/15
664/664 [==============================] - 2s 3ms/step - loss: 0.6517 - acc: 0.6160 - val_loss: 0.6405 - val_acc: 0.6622
Epoch 7/15
664/664 [==============================] - 2s 3ms/step - loss: 0.6497 - acc: 0.6235 - val_loss: 0.6414 - val_acc: 0.6622
Epoch 8/15
664/664 [===========

In [ ]:
model.save('Task_binary_classification_without_transfer_learning.h5')